# Deploy Object Detection Model Use ModelCI

MMDetction is a well-known open source object detection toolbox based on PyTorch. You can refer to <https://arxiv.org/abs/1906.07155> for more details.

By walking through this tutorial, you will be able to:

- Load pretained MMDetction model
- Convert MMDetction model into ONNX format 
- Register and retrieve models by ModelHub

 ## 1. Prequisities
 
 ### 1.1 Installation of MMDetction
 
 Firstly you have to install MMDetction according to official instructions : <https://mmdetection.readthedocs.io/en/latest/get_started.html#installation> 

In [1]:
!pip install https://download.openmmlab.com/mmcv/dist/latest/torch1.7.0/cu102/mmcv_full-latest%2Btorch1.7.0%2Bcu102-cp37-cp37m-manylinux1_x86_64.whl
!git clone https://github.com/open-mmlab/mmdetection.git
!cd mmdetection && pip install -q -r requirements/build.txt && pip install -q -v -e .

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Obtaining file:///home/sherry/project/github/ML-Model-CI/example/notebook/mmdetection
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.7.0
    Uninstalling mmdet-2.7.0:
      Successfully uninstalled mmdet-2.7.0
  Running setup.py develop for mmdet


### 1.2 Start ModelCI Service
Then we can start our ModelCI service, you should at least set mongodb password before starting. You can refer to [last notebook](https://github.com/cap-ntu/ML-Model-CI/blob/master/example/notebook/image_classification_model_deployment.ipynb) for more details.

In [2]:
%env MONGO_PASSWORD=modelci@2020
!conda activate modelci && modelci start

env: MONGO_PASSWORD=modelci@2020
2020-12-24 09:56:20,119 - ml-modelci Docker Container Manager - INFO - Container name=mongo-99022 stared
2020-12-24 09:56:21,831 - ml-modelci Docker Container Manager - INFO - Container name=cadvisor-75216 started.
2020-12-24 09:56:26,296 - ml-modelci Docker Container Manager - INFO - Container name=dcgm-exporter-29627 started.
2020-12-24 09:56:28,282 - ml-modelci Docker Container Manager - INFO - gpu-metrics-exporter-28682 stared
2020-12-24 09:56:28,571 - modelci backend - INFO - Uvicorn server listening on 8000


## 2. Build MMdetection Model
### 2.1 Imports
We should import the following functions:
- preprocess_example_input: for generating tensor and meta info from example image file
- build_model_from_cfg: for building model form config file and checkpoint file

In [3]:
from mmdet.core import preprocess_example_input, build_model_from_cfg

### 2.2 Model Config

We should either use a dict or config file for configuration of MMDetection model, to make things simple, we use a config file provided by MMDetection.

Notice: 

- You may need to manually download pretrained model checkpoints from [MMDetection models zoo](https://github.com/open-mmlab/mmdetection/blob/master/docs/model_zoo.md).
- Only a few MMdet models are able to converted into ONNX format, you can refer to [documentation](https://mmdetection.readthedocs.io/en/latest/tutorials/pytorch2onnx.html#list-of-supported-models-exportable-to-onnx) for more detail.

In [4]:
config_file = 'mmdetection/configs/retinanet/retinanet_r50_fpn_1x_coco.py'
checkpoint_file = 'retinanet_r50_fpn_1x_coco_20200130-c2398f9e.pth'

### 2.3 Build Model
Then we can build our MMdetection model based on the configuration above and the checkpoint file we already download.

In [5]:
model = build_model_from_cfg(config_file, checkpoint_file)

Before conversion, we need to modify forward function to provide the necessary **kwargs parameters such as img_metas.

In order to obtain valid bbox data during the onnx tracing process, we also need to use a tensor generated from image file as model input instead of random tensors.

In [6]:
input_config = {
    'input_shape': (1,3,224,224),
    'input_path': 'mmdetection/demo/demo.jpg',
    'normalize_cfg': {
        'mean': (123.675, 116.28, 103.53),
        'std': (58.395, 57.12, 57.375)
        }
}
one_img, one_meta = preprocess_example_input(input_config)

In [7]:
one_img.shape

torch.Size([1, 3, 224, 224])

In [8]:
from functools import partial
model.forward = partial(model.forward, img_metas=[[one_meta]], return_loss=False)

## 3. Register Model
We can convert the pytorch model above into optimized formats, such as ONNX through modelci

### 3.1 Imports
- modelci.hub.manager: for registering model into ModelHub
- modelci.types.bo: for constructing model inputs paramenters
- modelci.types.trtis_objects: for specifying model input shape format

In [9]:
from modelci.hub.manager import register_model
from modelci.types.bo import IOShape, Framework, Task, Engine, Metric, ModelVersion
from modelci.types.trtis_objects import ModelInputFormat

### 3.2 Sepcify Input and Output

Here are some parameters need to be specified before model conversion.
- inputs: The model inputs info
- outputs: The model outputs info


In [10]:
inputs = [IOShape([-1, 3, 204, 204], dtype=float, name='IMAGE', format=ModelInputFormat.FORMAT_NCHW)]
outputs = [
    IOShape([-1, 100, 5], dtype=float, name='BBOX'),
    IOShape([-1, 100], dtype=float, name='SCORE')
    ]

### 3.3 Register

In this step, the inputs and outputs variable generated before are reused. 

In [11]:
register_model(
    model,
    dataset='COCO',
    task=Task.OBJECT_DETECTION,
    metric={Metric.MAP: 0.365},
    outputs=outputs,
    inputs=inputs,
    architecture='RetinaNet',
    framework=Framework.PYTORCH,
    version=ModelVersion('1'),
    model_input=[one_img],
    profile=False
)

2021-01-13 21:54:15,831 - converter - WARNING - This model is not supported as torchscript format
2021-01-13 21:54:23,234 - converter - INFO - ONNX format converted successfully


As we could see, MLModelCI support auto conversion of PyTorch models into both torchscript and ONNX format, as a result.

However, this model cannot be transformed into torchscript format, but supportive of ONNX format conversion, there could be serveral factors contributing to model conversion failture such as the model structure and code format.

## 4. Retrieve Model

The following steps will retrieve the model we just registered.

In [12]:
from modelci.hub.manager import retrieve_model
retrieved_models = retrieve_model(
        architecture_name = 'RetinaNet',
        framework = Framework.PYTORCH,
        version=ModelVersion('1')
)

In [13]:
retrieved_models

It's no wonder we get two model objects here cause there is an addition ONNX format model created automatically during previous registering process.

In [14]:
retrieved_models[0].__dict__

{'_id': '5ffefb92681bb7cb565e3465',
 'name': 'RetinaNet',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.ONNX: 3>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7fb614070450>,
 'dataset': 'COCO',
 'metric': {<Metric.MAP: 1>: 0.365},
 'task': <Task.OBJECT_DETECTION: 1>,
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7fb614070590>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7fb614070610>,
 'weight': <modelci.types.bo.model_objects.Weight at 0x7fb5f8b3ed50>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'shanshan',
 'create_time': datetime.datetime(2021, 1, 13, 21, 54, 5, 916000)}

In [15]:
retrieved_models[1].__dict__

{'_id': '5ffefb91681bb7cb565e321b',
 'name': 'RetinaNet',
 'framework': <Framework.PYTORCH: 1>,
 'engine': <Engine.PYT: 6>,
 'version': <modelci.types.bo.model_objects.ModelVersion at 0x7fb5d231d850>,
 'dataset': 'COCO',
 'metric': {<Metric.MAP: 1>: 0.365},
 'task': <Task.OBJECT_DETECTION: 1>,
 'inputs': [<modelci.types.bo.model_objects.IOShape at 0x7fb61408d350>],
 'outputs': [<modelci.types.bo.model_objects.IOShape at 0x7fb61408d950>,
 'weight': <modelci.types.bo.model_objects.Weight at 0x7fb5f8b78f10>,
 'profile_result': None,
 'status': <Status.UNKNOWN: 0>,
 'creator': 'shanshan',
 'create_time': datetime.datetime(2021, 1, 13, 21, 54, 5, 916000)}